# yolov5 flask+aws  in window
배포 예제 출처:https://intrepidgeeks.com/tutorial/built-using-aws-lambdas-object-detection-api-flash-yllov5

# 1.configure 하기위한 템플릿 파일 이름변경

In [ ]:
!move conf.json.template conf.json

In [6]:
# conf 파일 open
import json
import pprint

with open('conf.json') as f:
    conf = json.load(f)

pprint.pprint(conf)

{'domain': 'XXXX.dkr.ecr.us-east-1.amazonaws.com',
 'function_name': 'lambda-container-yolov5-function-XXXX',
 'profile': 'produser',
 'repository_name': 'lambda-container-yolov5-XXXX',
 'role_arn': 'arn:aws:iam::XXXX:role/service-role/lambda-container-yolov5-role-XXXX',
 'tag': ':latest'}


# 2. conf.json 파일 수정
conf.json의 proflie key값을 현재 자신의 aws CLI profile name 으로 변경

In [2]:
conf["profile"] = "your name"

NameError: name 'conf' is not defined

# 3. container make
현재시간값을 받아 이름정해주기

In [7]:
import boto3, datetime, subprocess, json
from time import sleep

from boto3.session import Session
profile = conf["profile"]
session = Session(profile_name=profile)

In [8]:
now = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9))).strftime('%Y%m%d%H%M%S')
image = 'lambda-container-yolov5'
tag = ':latest'
repository_name = f'{image}-{now}'

## 4. ECR 에 boto3를 이용하여 푸시


In [1]:
%%time
# 컨테이너 생성
ecr = session.client('ecr')
response = ecr.create_repository(
    repositoryName=repository_name,
    imageScanningConfiguration={'scanOnPush': True},
)

NameError: name 'session' is not defined

In [11]:

uri = response['repository']['repositoryUri']
account_id = response['repository']['registryId']
region = uri.split('.')[3]
domain = uri.split('/')[0]

In [12]:
%%time

# ecr 푸시


!docker build -t {repository_name} --platform=linux/amd64 .


!docker tag {repository_name}{tag} {uri}{tag}


! aws ecr get-login-password --profile {profile} | docker login --username AWS --password-stdin {domain}


!docker push {uri}{tag}

#1 [internal] load build definition from Dockerfile
#1 sha256:23431ec3392664f02abbc1296bf407afba5fa266288dcb75f76861473b7b12d5
#1 transferring dockerfile: 760B done
#1 DONE 0.0s

#2 [internal] load .dockerignore
#2 sha256:cf67924232b77f67493e561ef8c886da8f814adff7517ae8b05bc59f868f94fb
#2 transferring context: 2B done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/library/python:3.8-slim-buster
#3 sha256:a82ddfb0a3c3ab3f4e2ebc7582cec39f26df7d1ae41d54f70ea9fe596d7b25c7
#3 DONE 4.1s

#4 [ 1/14] FROM docker.io/library/python:3.8-slim-buster@sha256:ed38d1a70cd28a628a58cecaa87bb95dba7180fffc0afd314f9fbcddce40c749
#4 sha256:4d8c4b0fe64fb7380bfd98833c1d207c4998d90235a28f370fcdba55019c9d26
#4 resolve docker.io/library/python:3.8-slim-buster@sha256:ed38d1a70cd28a628a58cecaa87bb95dba7180fffc0afd314f9fbcddce40c749 done
#4 sha256:ed38d1a70cd28a628a58cecaa87bb95dba7180fffc0afd314f9fbcddce40c749 1.86kB / 1.86kB done
#4 sha256:127a61592d0e06252c233ab56f2d832606daeca6857183c78e40ee3aaabb6e0d 

#6 8.644 Preparing to unpack .../11-libedit2_3.1-20181209-1_amd64.deb ...
#6 8.649 Unpacking libedit2:amd64 (3.1-20181209-1) ...
#6 8.692 Selecting previously unselected package libwayland-server0:amd64.
#6 8.692 Preparing to unpack .../12-libwayland-server0_1.16.0-1_amd64.deb ...
#6 8.697 Unpacking libwayland-server0:amd64 (1.16.0-1) ...
#6 8.734 Selecting previously unselected package libgbm1:amd64.
#6 8.735 Preparing to unpack .../13-libgbm1_18.3.6-2+deb10u1_amd64.deb ...
#6 8.739 Unpacking libgbm1:amd64 (18.3.6-2+deb10u1) ...
#6 8.776 Selecting previously unselected package libglapi-mesa:amd64.
#6 8.776 Preparing to unpack .../14-libglapi-mesa_18.3.6-2+deb10u1_amd64.deb ...
#6 8.781 Unpacking libglapi-mesa:amd64 (18.3.6-2+deb10u1) ...
#6 8.822 Selecting previously unselected package libwayland-client0:amd64.
#6 8.822 Preparing to unpack .../15-libwayland-client0_1.16.0-1_amd64.deb ...
#6 8.827 Unpacking libwayland-client0:amd64 (1.16.0-1) ...
#6 8.861 Selecting previously unselecte

Login Succeeded
The push refers to repository [432867094756.dkr.ecr.us-east-2.amazonaws.com/lambda-container-yolov5-20220531144532]
aeda6c719b99: Preparing
37bf2ef8ae81: Preparing
e4da67294981: Preparing
5f70bf18a086: Preparing
891940643a05: Preparing
9aea818f0fe4: Preparing
c348bfa25b30: Preparing
5e73d74ff98b: Preparing
055088f1f474: Preparing
77f636ede05a: Preparing
472465885c0c: Preparing
2f5e333d29d7: Preparing
3d45c289e33e: Preparing
6ab613504b6a: Preparing
f99d78fd828f: Preparing
429ac8006c35: Preparing
4bdae028fbe3: Preparing
10e6bc6fdee2: Preparing
6ab613504b6a: Waiting
f99d78fd828f: Waiting
055088f1f474: Waiting
10e6bc6fdee2: Waiting
77f636ede05a: Waiting
429ac8006c35: Waiting
4bdae028fbe3: Waiting
2f5e333d29d7: Waiting
3d45c289e33e: Waiting
c348bfa25b30: Waiting
5e73d74ff98b: Waiting
9aea818f0fe4: Waiting
aeda6c719b99: Pushed
5f70bf18a086: Pushed
e4da67294981: Pushed
37bf2ef8ae81: Pushed
891940643a05: Pushed
055088f1f474: Pushed
9aea818f0fe4: Pushed
5e73d74ff98b: Pushed
4724

In [14]:

res = ecr.describe_images(
    repositoryName = repository_name
)
image_digest = res['imageDetails'][-1]['imageDigest']

## Lambda 역할, 정책 그리고 연결까지


In [28]:
iam = session.client('iam')
function_name = f'{image}-function-{now}'
doc = {
    'Version': '2012-10-17',
    'Statement': [
        {
            'Action': 'sts:AssumeRole',
            'Principal': {
                'Service': 'lambda.amazonaws.com'
                
            },
            'Effect': 'Allow',
            'Sid': ''
            
        }
    ]
}

#역할
role_name = f'{image}-role1-{now}'
res = iam.create_role(
    Path = '/service-role/',
    RoleName=role_name,
    AssumeRolePolicyDocument=json.dumps(doc),
    Description=f'exec role',
    MaxSessionDuration=3600*12
)
role_arn = res['Role']['Arn']

doc = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "logs:CreateLogGroup",
            "Resource": f"arn:aws:logs:{account_id}:*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": [
                f"arn:aws:logs:{region}:{account_id}:log-group:/aws/lambda/{function_name}:*"
            ]
        }
    ]
}

#정책생성

poicy_name = f'{image}-policy-{now}'
res = iam.create_policy(
    PolicyName=poicy_name,
    PolicyDocument=json.dumps(doc),
)
policy_arn = res['Policy']['Arn']
# 생성한 것 attach

res = iam.attach_role_policy(
    RoleName=role_name,
    PolicyArn=policy_arn
)

sleep(20)

## Lambda 함수작성


In [33]:
%%time
# lambda 함수를 도커이미지를 사용하여 작성
lambda_client = session.client('lambda')
res = lambda_client.create_function(
    FunctionName=function_name,
    Role=role_arn,
    Code={
        'ImageUri': f'{uri}@{image_digest}'
    },
    Description='input-> b64img, output -> b64img, yolov5 detect',
    Timeout=60*15,
    MemorySize=1024,
    Publish=True,
    PackageType='Image',
)

while True:
    res = lambda_client.get_function(FunctionName=function_name)
    try:
        if res['Configuration']['StateReasonCode']=='Creating':
            print('.',end='')
            sleep(1)
    except:
        if res['Configuration']['LastUpdateStatus']=='Successful':
            print('!')
            break
        else:
            print('?')
            break


..............................................................!
Wall time: 1min 18s


## 추론(inference)

In [53]:
import base64
import requests
import cv2
import numpy as np
from matplotlib import pyplot as plt
input_file = './yolov5/data/images/bul18.jpg'
data = {}
with open(input_file,'rb') as f:
    data['body']= base64.b64encode(f.read()).decode('utf-8')

In [41]:
%%time
res = lambda_client.invoke(
    FunctionName=function_name,
    Payload=json.dumps(data)
)

payload = res["Payload"]

body = payload.read()
body = body.decode()
json_dict = json.loads(body)
print(json_dict)

{'statusCode': 200, 'body': '[{"xmin":821.3532104492,"ymin":425.091003418,"xmax":1023.7008666992,"ymax":704.936706543,"confidence":0.6528223157,"class":0,"name":"buliding"},{"xmin":0.3871765137,"ymin":134.0449523926,"xmax":155.2770996094,"ymax":638.2137451172,"confidence":0.394326359,"class":0,"name":"buliding"}]'}
Wall time: 8.54 s


# ---------------------------------
아래 코드는 api작업 이후 사용

In [ ]:
!curl --data-binary @fl2.jpg  https://mz1gdes0a3.execute-api.us-east-2.amazonaws.com/dev/detect

## 삭제하기

In [ ]:
lambda_client.delete_function(
    FunctionName=function_name
)

In [ ]:
iam.detach_role_policy(
    RoleName=role_name,
    PolicyArn=policy_arn
)

In [ ]:
iam.delete_policy(
    PolicyArn=policy_arn
)

In [ ]:
iam.delete_role(
    RoleName=role_name,
)

In [ ]:
ecr.batch_delete_image(
    repositoryName=repository_name,
    imageIds=[{
        'imageDigest':image_digest
    }]
)

In [ ]:
ecr.delete_repository(
    repositoryName=repository_name,
)